## LSTM (long short-term memory) based Recurrent NeuralNet for share price forecasting
###### Abdulla Al Blooshi
----------------
- As an overview, the idea behind this model architecture is to assign weights to selected features; in this case the **open** and the **highest** price the stock reached for a given day were selected. These learned weights represents the model's view on the importance of said features from recent and previous time blocks and how they affect the price in the coming day(s).
- This model will be trained on ADNOC's stock price history and the weights it learns will be saved and used via a transfer learning approach to be tested on and predict Borouge's future stock price.
- This was done due to the lack of training data for Borouge's stock price as it (relatively) recently IPO'd
<br>
<br>
> *This is by no means financial advice as I am not a financial expert, and all the data used here is publicly available.*

In [26]:
import numpy as np 
import pandas as pd 
import tensorflow.keras as keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    LSTM
)
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#### Splitting and normalizing the data:
   - It is easier for the model to work with numbers that are closer together; namely in the range of (0,1) for our case
   - The `MinMaxScaler()` transformation is given by:
      
        X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))<br>
        X_scaled = X_std * (max - min) + min <br>
        
        *where min, max = feature_range.*
        > taken straight from the docs

In [27]:
unsplit_dat = pd.read_csv('./data/ADNOCDIST_Historical_Data.csv', index_col='Date')
valid_dat = pd.read_csv('./data/ADNOC_Valid.csv', index_col='Date')


total_set = unsplit_dat[['Open']].values

# *Shuffle set to false for smaller dataset to avoid sampling bias, default behavior is shuffle first then split
training_set, test_set = train_test_split(total_set, test_size=0.02, random_state=44, shuffle=False)

valid_set = valid_dat[['Open']].values

normalizer = MinMaxScaler(feature_range=(0,1))

scaled_train_set = normalizer.fit_transform(training_set)
scaled_valid_set = normalizer.fit_transform(valid_set)
# scaled_test_set = normalizer.fit_transform(test_set)
# scaled_total_set = normalizer.fit_transform(total_set)

print(f"Training set\n{scaled_train_set[:10]}\nValidation set\n{scaled_valid_set[:10]}\nTest set\n{test_set[:10]}")
training_set.shape

Training set
[[0.70333333]
 [0.70333333]
 [0.69333333]
 [0.69      ]
 [0.68666667]
 [0.69333333]
 [0.69      ]
 [0.68666667]
 [0.70666667]
 [0.70333333]]
Validation set
[[0.8       ]
 [0.8       ]
 [0.9       ]
 [0.86666667]
 [0.9       ]
 [0.73333333]
 [0.7       ]
 [0.93333333]
 [0.83333333]
 [0.8       ]]
Test set
[[2.64]
 [2.61]
 [2.62]
 [2.62]
 [2.65]
 [2.62]
 [2.62]
 [2.6 ]
 [2.6 ]
 [2.62]]


(1023, 1)

#### LSTMs distinguishing features and other important notes:
- As per the [paper](http://www.bioinf.jku.at/publications/older/2604.pdf) first proposing this architecture by Hochreiter et al. Constant error carousels (CEC) are the central features of LSTMs. Controlling (deciding) the backward propagation of errors through the network.
    - These CEC's are then extended to form what is referred to as a memory cell; the extension adds multiplicative input and output gates. These gates control the contents with in a cell from being propagated and control the cell from activating other units respectively.
- RNNs are able to use recently seen previous information and cannot do so with information with larger time lags between them, this is where the LSTM architecture comes into play.
- This is another one of the distinguishing features of using the LSTM architecture, its ability to 'remember' or erase parts of previously seen data in a window (or timestep).
- By creating a window our training data will be turned into an array of arrays divided into chunks of N, where N would be the size of our timestep/window.
    - for example having N be 60 would allow our model to use the previous sixty days of data to make the prediction for the 61st.


In [28]:
# Bismillah
#TODO: y_train should predict close prices?
#TODO: Consider using sklearn's standardScaler()
X_train=[]
y_train=[]
X_valid=[]
y_valid=[]

for i in range(60,scaled_train_set.shape[0]):
    X_train.append(scaled_train_set[i-60:i,0])
    y_train.append(scaled_train_set[i,0])

for j in range(60,scaled_valid_set.shape[0]):
    X_valid.append(scaled_valid_set[j-60:j,0])
    y_valid.append(scaled_valid_set[j,0])

# Keras accepts numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

X_valid, y_valid = np.array(X_valid), np.array(y_valid) 
print(X_train.shape, y_train.shape)

(963, 60) (963,)


#### Implementation Notes:
- The model base architecture will be sequential which ["_groups a linear stack of layers into a tf.keras.Model_"](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
- Ideally I would've liked to add more dropout layers but I do not have that luxury as data is limited. The dropout layers aids in reducing the amount of overfitting.

___However, before we can do that the data must be transformed further into a 3D array with X_train learning examples. In our case it will be of dimension (984,60,1), the 984 comes from the number of samples we have, the 60 is because we grouped our samples into groups of 60, and the 1 is because we want the model to access one feature at each timestep___ 


In [29]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_valid = np.reshape(X_valid,(X_valid.shape[0],X_valid.shape[1],1))

In [30]:
X_train.shape

(963, 60, 1)

In [31]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=32, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.1))
lstm_model.add(LSTM(units=32, return_sequences=True))
lstm_model.add(Dropout(0.05))
lstm_model.add(LSTM(units=32))
lstm_model.add(Dropout(0.05))
lstm_model.add(Dense(units=1))

lstm_model.summary()
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 60, 32)            4352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 32)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 60, 32)            8320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 32)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

#### Training the model on the data:

In [33]:
lstm_model.fit(X_train, y_train, epochs=100,batch_size=32,validation_data=(X_valid,y_valid))

Epoch 1/100
31/31 [==============================] - 4s 117ms/step - loss: 0.0031 - val_loss: 0.0282
Epoch 2/100
31/31 [==============================] - 2s 59ms/step - loss: 0.0024 - val_loss: 0.0279
Epoch 3/100
31/31 [==============================] - 2s 59ms/step - loss: 0.0023 - val_loss: 0.0280
Epoch 4/100
31/31 [==============================] - 2s 59ms/step - loss: 0.0024 - val_loss: 0.0272
Epoch 5/100
31/31 [==============================] - 2s 60ms/step - loss: 0.0024 - val_loss: 0.0279
Epoch 6/100
31/31 [==============================] - 2s 58ms/step - loss: 0.0024 - val_loss: 0.0268
Epoch 7/100
31/31 [==============================] - 2s 58ms/step - loss: 0.0021 - val_loss: 0.0312
Epoch 8/100
31/31 [==============================] - 2s 58ms/step - loss: 0.0024 - val_loss: 0.0263
Epoch 9/100
31/31 [==============================] - 2s 60ms/step - loss: 0.0023 - val_loss: 0.0271
Epoch 10/100
31/31 [==============================] - 2s 59ms/step - loss: 0.0023 - val_loss: 0.026

#### Testing the model on unseen data

- After training the validation loss is reasonably larger than loss on the training set, hinting towards the possibility of underfitting 
- Possible solutions are:
    - Parameter increase
    - Additional layers of
    - Increased training set size

In [34]:
%who

Dense	 Dropout	 LSTM	 MinMaxScaler	 SGD	 Sequential	 X_test	 X_train	 X_valid	 
e	 i	 j	 keras	 lstm_model	 model_prediction	 normalizer	 np	 os	 
pd	 scaled_train_set	 scaled_valid_set	 sys	 test_set	 tests	 total_set	 train_test_split	 training_set	 
unsplit_dat	 valid_dat	 valid_set	 y_train	 y_valid	 


In [35]:
# len(scaled_total_set)-len(scaled_test_set)-60 = 963 = X_train.shape[0], the remaining being test
tests=total_set[len(total_set)-len(test_set)-60:]
tests = tests.reshape(-1,1)
# tests = tests.reshape(-1,1)
tests = normalizer.transform(tests)

X_test=[]
# Creating the same window and dimensions as before
for e in range(60,tests.shape[0]):
    X_test.append(tests[e-60:e,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

model_prediction = lstm_model.predict(X_test)
model_prediction = normalizer.inverse_transform(model_prediction)
model_prediction

array([[3.509372 ],
       [3.5098052],
       [3.5101452],
       [3.5103612],
       [3.5105457],
       [3.5106266],
       [3.5105953],
       [3.5104556],
       [3.5103636],
       [3.510312 ],
       [3.5103216],
       [3.5103247],
       [3.5103552],
       [3.510565 ],
       [3.5107515],
       [3.5108151],
       [3.5106323],
       [3.5100656],
       [3.5091448],
       [3.5082622],
       [3.5077653]], dtype=float32)

<ul style="color:orange">
<li>TODO: ExponMovingAvg for comparison</li>
<li>TODO: Need to resolve package conflicts with matplotlib, and plot results nicely...</li>
<li>TODO: Save model and use transfer learning to apply to Borouge stock</li>
</ul>